# 读取和存储

到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


## 4.5.1 load and save NDarray

我们可以直接使用`save`函数和`load`函数分别存储和读取。下面的例子创建了tensor`x`，并将其存在文件名同为`x`的文件里。

In [2]:
import numpy as np

x = tf.ones(3)
x

<tf.Tensor: id=2, shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

然后我们将数据从存储的文件读回内存。

In [3]:
np.save('x.npy', x)
x2 = np.load('x.npy')
x2

array([1., 1., 1.], dtype=float32)

我们还可以存储一列`tensor`并读回内存。

In [4]:
y = tf.zeros(4)
np.save('xy.npy',[x,y])
x2, y2 = np.load('xy.npy', allow_pickle=True)
(x2, y2)

(<tf.Tensor: id=6, shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Tensor: id=7, shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

我们甚至可以存储并读取一个从字符串映射到`tensor`的字典。

In [5]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: id=8, shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'y': <tf.Tensor: id=9, shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 4.5.2 load and save model parameters

我们还可以读写模型的参数。
为了演示方便，我们先创建一个多层感知机，并将其初始化。。

In [6]:
X = tf.random.normal((2,20))
X

<tf.Tensor: id=15, shape=(2, 20), dtype=float32, numpy=
array([[ 1.4838032 , -0.638382  , -0.7836789 ,  1.8679693 , -0.73148364,
        -0.12649764, -0.2709544 , -0.33071974,  0.08754155, -0.11141171,
         0.18274567, -0.64928424, -0.6519136 ,  0.07320689, -0.5973234 ,
         1.9181312 ,  0.47066143, -0.10463867, -0.48717928,  0.3107364 ],
       [ 0.37838233,  0.11170077, -1.3378098 ,  0.3618399 ,  0.27140674,
         0.9901546 ,  1.4799279 ,  1.2373866 , -0.62953895, -1.5107338 ,
        -1.6658096 , -0.08139827,  0.5444429 ,  0.94359463, -0.00676966,
        -1.5311289 , -0.30671307,  0.38309866, -0.2765001 , -0.61528987]],
      dtype=float32)>

In [7]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

net = MLP()
Y = net(X)
Y

<tf.Tensor: id=71, shape=(2, 10), dtype=float32, numpy=
array([[-0.30077258,  0.4493576 ,  0.00761353, -0.14657806, -0.11702831,
        -0.20244044,  0.15949515, -0.025849  , -0.36856648,  0.23903428],
       [-0.09660852,  0.0096112 ,  0.3435048 , -0.066409  , -0.24335058,
        -0.01852736,  0.77680373, -0.04183513, -0.232623  , -0.5856861 ]],
      dtype=float32)>

下面把该模型的参数存成文件，文件名为`4.5saved_model.h5`。

In [8]:
net.save_weights("4.5saved_model.h5")

接下来，我们再实例化一次定义好的多层感知机。与随机初始化模型参数不同，我们在这里直接读取保存在文件里的参数。

因为这两个实例都有同样的模型参数，那么对同一个输入X的计算结果将会是一样的。我们来验证一下。

In [9]:
net2 = MLP()
net2(X)
net2.load_weights("4.5saved_model.h5")
Y2 = net2(X)
Y2 == Y

<tf.Tensor: id=146, shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>